In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import riiideducation
env = riiideducation.make_env()

# Training data is in the competition dataset as usual
iter_test = env.iter_test()
set_predict = env.predict

**Some preprocessing and selecting of data to feed to the collaborative filetering- Using the last 50 questions of every user, hoping to catch their current understanding and level.**

In [ ]:
import pandas as pd
dtypes = {
    "row_id": "uint32",
    "timestamp": "float32",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float16", 
    "prior_question_had_explanation": "boolean"
}

data = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes)
data.drop(columns=['timestamp','user_answer'],inplace=True)
print("Train size:", data.shape)
import numpy as np
import gc
import time
import fastai
from fastai.tabular.all import *
from fastai.collab import *

fastai.__version__


#MakeQuestions since Lecture
start = time.time()

npdat=data[['user_id','content_type_id','answered_correctly']].to_numpy()
npdat=np.append(npdat, np.zeros((npdat.shape[0],1),dtype=int), axis=1)
npdat=np.append(npdat, np.zeros((npdat.shape[0],1),dtype=int), axis=1)

for i in range(1,npdat.shape[0],1):
    if npdat[i-1,0]==npdat[i,0]:
        if npdat[i,1]==False :
            npdat[i,3]=(npdat[i-1,3] +1)
            npdat[i,4]=(npdat[i-1,3] * npdat[i-1,4] + npdat[i-1,2]*100)/npdat[i,3]
        else:
            npdat[i,3]=0
            npdat[i,4]=0
    else:
        if npdat[i,1]==False :
            npdat[i,3]=1
            npdat[i,4]=0
        else:
            npdat[i,3]=0
            npdat[i,4]=0
data['qsincel']=npdat[:,3].astype(np.uint8)
data['avgcsincel']=npdat[:,4].astype(np.uint8)
del npdat
gc.collect()

print('Done qsincel in :',(time.time() - start) ,'s')


#Make Tasks since last lecture
start = time.time()
npa=data[['user_id','task_container_id','content_type_id']].to_numpy()
npa=np.append(npa, np.zeros((npa.shape[0],1) ,dtype=int), axis=1)

for i in range(1,npa.shape[0],1):
    if npa[i-1,0]==npa[i,0]:
        if npa[i,1]==npa[i-1,1]:
            if npa[i,2]==0:
                npa[i,3]=npa[i-1,3]
            else:
                 npa[i,3]=0
        else:
            if npa[i,2]==0:
                npa[i,3]=npa[i-1,3]+1
            else :
                npa[i,3]=0
    else:
        if npa[i,2]==0 :
            npa[i,3]= 1
        else:
            npa[i,3]= 0         
data['tsincel']=npa[:,3].astype(np.uint8)
del npa
gc.collect()
print('Done tsincel in :',(time.time() - start) ,'s')

# dls = CollabDataLoaders.from_df(data[['user_id','content_id','answered_correctly']],item_name='content_id',valid_pct=0,bs=2048).cuda()
# learn1 = collab_learner(dls, n_factors=3, y_range=(0, 1),use_nn= True).to_fp16()
# learn1.fit_one_cycle(1, 5e-3,wd=1e-1)

#divides the questions


from sklearn.decomposition import PCA


questions = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")

lst = []
for tags in questions["tags"]:
    ohe = np.zeros(188)
    if str(tags) != "nan":
        for tag in tags.split():
            ohe += np.eye(188)[int(tag)]
    lst.append(ohe)
tags_df = pd.DataFrame(lst, columns=[f"tag_{i}" for i in range(188)]).astype(int)
del lst,ohe
#WITH PCA
# from sklearn.cluster import KMeans,MiniBatchKMeans
# import numpy as np
# pca = PCA(n_components=35)
# b=pca.fit_transform(tags_df.values)

# q=pd.DataFrame(b)
# q['content_id']=q.index
# #tda=data.iloc[int(9e7):,3]
# tda=data.sample(n=int(3e6))['content_id']
# tdat=pd.merge(tda,q,how='left',on='content_id').dropna().iloc[:,1:36]
# nptda=tdat.to_numpy()
# kmeans = MiniBatchKMeans(n_clusters=5,
#                           random_state=0,
#                           batch_size=256).fit(nptda)
# del tdat,nptda,tda,tags_df,b
# gc.collect()
# npq= q.iloc[:,0:35]
# q['group']=kmeans.predict(npq).astype(np.int8)
# rq=q.iloc[:,35:]
# dat=pd.merge(data[data['content_type_id']==False],rq,how='left',on='content_id',copy=False)
# del npq,q,data,kmeans,rq,questions
# gc.collect()


#WITHOUT PCA
tags_df['content_id']=tags_df.index
from sklearn.cluster import KMeans,MiniBatchKMeans
# tda=data.iloc[int(8e7):,3]
tda=data.sample(n=int(3e6))['content_id']
tdat=pd.merge(tda,tags_df,how='left',on='content_id').dropna().iloc[:,1:]
nptda=tdat.to_numpy()
kmeans = MiniBatchKMeans(n_clusters=50,reassignment_ratio=0.09,
                          random_state=1,
                          batch_size=256).fit(nptda)
del tdat,nptda,tda
gc.collect()
npq= tags_df.iloc[:,0:188]
tags_df['group']=kmeans.predict(npq).astype(np.uint8)
rq=tags_df.loc[:,['group','content_id']]

#Merge to get Question Groups 

train=pd.merge(data[data['content_type_id']==False],rq,how='left',on='content_id')
del npq,data ,tags_df 
gc.collect()

batchsize= int(5e6)

train['mobile_mean6']= np.zeros(train.shape[0], dtype=np.uint8)
train['mobile_mean9']= np.zeros(train.shape[0], dtype=np.uint8)
train['mobile_mean12']= np.zeros(train.shape[0], dtype=np.uint8)


rn=0
done=0
print('Calculating mean, while dividing into batches')
while done ==0:
    n= min(rn+batchsize,train.shape[0]-1)
    us=train.loc[n,'user_id']
    cutoff= train[train['user_id']==us].reset_index()['index'].max()
    print('starting on ', train.loc[rn:cutoff].shape,'data' )
    train.loc[rn:cutoff,'mobile_mean9'] = train.loc[rn:cutoff].groupby('user_id').shift(+1).rolling(9)['answered_correctly'].mean()*100
    train.loc[rn:cutoff,'mobile_mean6'] = train.loc[rn:cutoff].groupby('user_id').shift(+1).rolling(6)['answered_correctly'].mean()*100
    train.loc[rn:cutoff,'mobile_mean12'] = train.loc[rn:cutoff].groupby('user_id').shift(+1).rolling(12)['answered_correctly'].mean()*100

    

    
    rn=cutoff+1
    if rn>=train.shape[0]:
        print('end')
        done=1
    
    
        
# train['mobile_mean6'].fillna(0.6,inplace= True)
# train['mobile_mean6']=(train['mobile_mean6']*100).astype(np.int8)

train['mobile_mean9'].fillna(0.6,inplace= True)
# train['mobile_mean9']=(train['mobile_mean9']*100).astype(np.int8)
train['mobile_mean6'].fillna(0.6,inplace= True)
# train['mobile_mean6']=(train['mobile_mean6']*100).astype(np.int8)
train['mobile_mean12'].fillna(0.6,inplace= True)

A=train.groupby('user_id').tail(30).copy()
del train
gc.collect()
train= A

lastrec= train.groupby('user_id').tail(1).copy()

tu=train['user_id'].unique().tolist()
tc=train['content_id'].unique().tolist()
questions['content_id']=questions['question_id']
train=pd.merge(train,questions.loc[:,['content_id','part']],how='left',on='content_id')



**COLLABORATICE FILTERING FAST AI, TRAIN MODEL **

In [ ]:
start = time.time()
nfact=3
cdls = CollabDataLoaders.from_df(train.loc[:,['user_id','content_id','answered_correctly']],
                            item_name='content_id',valid_pct=0,bs= 4096)#8192   16384
coll = collab_learner(cdls, n_factors=nfact, y_range=(-0.1, 1.1),wd=1e-1)
coll .fit_one_cycle(2, 5e-3)
print('Trained model total time :',(time.time() - start),'s')




**GETTING THE MODELS CREATED BIASES AND VARIABLES AND  CALCULATING PREDICTIONS MANUALLY- FASTER WAY THAN METHOD MODEL.GET_PREDICTS - WITCH IS TOO SLOW TO USE LATER ON THE PRIVATE TEST SET**

In [ ]:
train['collabPreds']= np.zeros(train.shape[0], dtype=np.int16)
train['collabUbias']= np.zeros(train.shape[0], dtype=np.int8)
train['collabCbias']= np.zeros(train.shape[0], dtype=np.int16)

start = time.time()

stepsize= int(5e6)+1 # how many samples i would like to really process at a time

print('Doing part ')
rn=0
done=0


collabPred,collabCbias,collabUbias = pd.Series(dtype=np.int8),pd.Series(dtype=np.int8),pd.Series(dtype=np.int8)

batch=train
u=batch.loc[:,'user_id'].to_numpy()
print(u.shape)
c=batch.loc[:,'content_id'].to_numpy()    
cb=coll.model.bias(c,is_item=True)
cw=coll.model.weight(c,is_item=True)   
#         print('content handeld')
ub=coll.model.bias(u,is_item=False)
print(ub.shape)
uw=coll.model.weight(u,is_item=False)
pre=torch.einsum('ij,in->i',cw,uw)
pre=pre

for x in range(0, nfact):
    train["User_weight{0}".format(x)] =uw[:,x].numpy()
    
collabPred=collabPred.append((pd.Series(pre)*1000000).astype(np.int16)) 
collabCbias=collabCbias.append((pd.Series(cb)*100).astype(np.int8)) 
collabUbias= collabUbias.append((pd.Series(ub)*100*50).astype(np.int16)) 

#         nwtestdf.loc[:,'qsincel']
#         a['row_id']=gdata.loc[rn:n,'row_id'].values
#         a['collabPreds']=collabPred.values
#         a['collabCbias']=collabCbias.values
#         a['collabUbias']=collabUbias.values
#         del collabPred , collabCbias , collabUbias ,u ,c,cb,cw, ub,uw,pre ,gdata
#         gc.collect()
#         train=pd.merge(train,a,how='left',on='row_id')
#         print('merged 10 m')

#         train.loc[train['group']==mn slice(rn:n) ,'collabPreds']=collabPred.values
#         train.loc[idx[train['group']==mn ],idx[rn:n,'collabPreds']]=collabPred.values
print(train.loc[:,'collabPreds'].shape,collabPred.values.shape)
train.loc[:,'collabPreds']=collabPred.values
train.loc[:,'collabUbias']=collabUbias.values
train.loc[:,'collabCbias']=collabCbias.values
print('step done results handeld, out of :',train.shape[0])

**TABULAR MODEL TO USE THE OUTPUTS OF THE COLLABORATIVE FILTERING TOGETHER WITH THE OTHER CREAATED DATA**

In [ ]:
train['prior_question_elapsed_time']=train['prior_question_elapsed_time'].clip(lower=0, upper=80000)
train['answered_correctly']=train['answered_correctly'].astype(np.bool_)
train['prior_question_had_explanation'].fillna(0,inplace=True)
train.dropna(inplace=True)

features=['prior_question_had_explanation',"answered_correctly",'task_container_id',
          'prior_question_elapsed_time',\
          'content_id',
        'collabPreds',
          'collabCbias','collabUbias',\
        'group','qsincel','avgcsincel','tsincel',\
          'User_weight0',
          'User_weight1',
          'User_weight2',
#         'mobile_mean3','Exmobile_mean3',#'mobile_mean3Diff',\
        'mobile_mean6',#'Exmobile_mean6',#'mobile_mean6Diff',\
        'mobile_mean9',#'Exmobile_mean9',#'mobile_mean9Diff',\
        'mobile_mean12',#'Exmobile_mean12',#'mobile_mean12Diff'\
         'part',]


dls = TabularDataLoaders.from_df(train[features],
    cat_names = ['prior_question_had_explanation','group',
                 #                 'mobile_mean3','Exmobile_mean3',#'mobile_mean3Diff',\
                'mobile_mean6',#'Exmobile_mean6',#'mobile_mean6Diff',\
                'mobile_mean9',#'mobile_mean9Diff',\
                'mobile_mean12',#'Exmobile_mean12',#'mobile_mean12Diff'\
                'part','content_id'],
    cont_names = ['task_container_id','prior_question_elapsed_time',\
                'collabPreds','collabCbias','collabUbias',
                  'qsincel','avgcsincel','tsincel',
                'User_weight0',
                  'User_weight1',
                  'User_weight2'
                 # 'Exmobile_mean9'
                 ],
    y_names="answered_correctly",
    bs=512,
    procs = [Categorify, FillMissing, Normalize])

# nn_learn = tabular_learner(ndls,metrics=accuracy,layers=[50,100,100],wd=1e-1)
# nn_learn.fit_one_cycle(2, lr=5e-2,pct_start=0.2)
learn = tabular_learner(dls, metrics=[accuracy,RocAucBinary()],layers=[100,200,200,100],wd=1e-1)
# learn = tabular_learner(dls,layers=[100,200,200,100],wd=1e-1)
learn.fit_one_cycle(1)
# print ('Trained NN')

lastrec= train.groupby('user_id').tail(1).copy()

tu=train['user_id'].unique().tolist()
tc=train['content_id'].unique().tolist()

kuserweights= coll.model.weight(lastrec['user_id'].to_numpy(),is_item=False)
lastrec.reset_index(drop=True,inplace=True)
lastrec['u_index_inrec']=lastrec.index

content_df=pd.DataFrame()
content_df['content_id']=train['content_id'].unique()
content_df['collabCbias']=(coll.model.bias(train['content_id'].unique(),is_item=True).numpy()*100).astype(np.int8)
kcontentweights= coll.model.weight(train['content_id'].unique(),is_item=True)
content_df.reset_index(drop=True,inplace=True)
content_df['c_index_inrec']=content_df.index

trainmedpred= train['collabPreds'].mean()
trainmedub= train['collabCbias'].mean()
trainmedcb= train['collabUbias'].mean()
trainmedtaskcont= train["task_container_id"].median()
trainmeduw0= train['User_weight0'].mean()
trainmeduw1= train['User_weight1'].mean()
trainmeduw2= train['User_weight2'].mean()

In [ ]:
%%time
#TESTING
# for i in range(1):
for (test, sample_prediction_df) in iter_test:        

    s=test.copy()
    if type(test)== pd.core.series.Series:
        test=pd.DataFrame(test).T
        
    test=test.loc[test['content_type_id']==0,:].copy()

    
#     test["row_id"]=test["row_id"].astype(np.uint32)
    test["timestamp"]=test["timestamp"].astype(np.uint32).fillna(0)
    test["user_id"]=test["user_id"].astype(np.uint32).fillna(0)
    test["content_id"]=test["content_id"].astype(np.uint32).fillna(0)
    test["content_type_id"]=test["content_type_id"].astype(np.uint32).fillna(0)
    test["task_container_id"]=test["task_container_id"].astype(np.uint32).fillna(trainmedtaskcont)
    test["prior_question_elapsed_time"]=test["prior_question_elapsed_time"].fillna(0).clip(lower=0, upper=80000).astype(np.uint32)
    test["prior_question_had_explanation"]=test["prior_question_had_explanation"].fillna(0).astype(np.uint32)
    
    #group & part
    test=pd.merge(test[test['content_type_id']==False],rq,how='left',on='content_id')
    test=pd.merge(test,questions.loc[:,['content_id','part']],how='left',on='content_id')
    # c_index_inrec
    test=pd.merge(test,content_df,how='left',on='content_id')
    # u_index_inrec
    test=pd.merge(test,lastrec.loc[:,['user_id','mobile_mean9','mobile_mean6','mobile_mean12','qsincel','tsincel','avgcsincel','collabUbias','u_index_inrec','User_weight0','User_weight1', 'User_weight2']],how='left',on='user_id')
    test.loc[:,'qsincel']= test.loc[:,'qsincel']+1
    test.loc[:,'tsincel']= test.loc[:,'tsincel']+1
    test.loc[:,'avgcsincel']= test.loc[:,'avgcsincel'].fillna(0) 
    test.loc[:,'qsincel']= test.loc[:,'qsincel'].fillna(0) 
    test.loc[:,'tsincel']= test.loc[:,'tsincel'].fillna(0)
    test.loc[:,'mobile_mean6']= test.loc[:,'mobile_mean6'].fillna(60)
    test.loc[:,'mobile_mean9']= test.loc[:,'mobile_mean9'].fillna(60)
    test.loc[:,'mobile_mean12']= test.loc[:,'mobile_mean12'].fillna(60)
    
    
    #create pre
    l=torch.Tensor(test.loc[test['u_index_inrec'].notnull()& test['c_index_inrec'].notnull(),['u_index_inrec']].u_index_inrec.values).to(dtype=torch.int64)
    kutest_weight=torch.index_select(kuserweights,0,l)
    l=torch.Tensor(test.loc[test['u_index_inrec'].notnull() & test['c_index_inrec'].notnull(),['c_index_inrec']].c_index_inrec.values).to(dtype=torch.int64)
    kcontent_weight=torch.index_select(kcontentweights,0,l)
    if len(kutest_weight) !=  0:         
        test.loc[test['u_index_inrec'].notnull() & test['c_index_inrec'].notnull(),['collabPreds']]=torch.einsum('ij,in->i',kutest_weight,kcontent_weight).numpy() *1000000
        test['collabPreds']=test['collabPreds'].fillna(trainmedpred).astype(np.int16)
    else:
        test['collabPreds']=trainmedpred.astype(np.int16)
    #elimnulls and amake sure of type
#     test['collabPreds']=test['collabPreds'].fillna(trainmedpred).astype(np.int16)
    test['collabCbias']=test['collabCbias'].fillna(trainmedcb).astype(np.int8)
    test['collabUbias']=test['collabUbias'].fillna(trainmedub).astype(np.int16)
    test['User_weight0']=test['User_weight0'].fillna(trainmeduw0).astype(np.int16)
    test['User_weight1']=test['User_weight1'].fillna(trainmeduw1).astype(np.int16)
    test['User_weight2']=test['User_weight2'].fillna(trainmeduw2).astype(np.int16)

    
    test.loc[:,'prior_question_elapsed_time']=test['prior_question_elapsed_time'].fillna (0)
    pfeatures = ['prior_question_had_explanation','task_container_id',
                    'prior_question_elapsed_time',\
                    'collabPreds','collabCbias','collabUbias',\
                    'group','qsincel','avgcsincel','tsincel',\
                     'User_weight0',
                      'User_weight1',
                      'User_weight2',
                 
    #                     'mobile_mean3','Exmobile_mean3',#'mobile_mean3Diff',\
                    'mobile_mean6',#'Exmobile_mean6',#'mobile_mean6Diff',\
                    'mobile_mean9',#'Exmobile_mean9',#'mobile_mean9Diff',\
                    'mobile_mean12',#'Exmobile_mean12',#'mobile_mean12Diff'\
                     'part','content_id']

    dl=dls.test_dl(test.loc[test['content_type_id']==0,pfeatures])

    #             print(learn.predict(X_test_value[pfeatures])[2][1].item())
    test.loc[test['content_type_id']==0,'answered_correctly']=(learn.get_preds(dl=dl,with_input=True, with_decoded=True)[1][:,1]).numpy()
    test['answered_correctly']=test['answered_correctly'].fillna(0.7)
    set_predict(test.loc[test['content_type_id'] == 0, ['row_id', 'answered_correctly']])